In [1]:
import os
import torch
from torchvision.transforms import transforms as T
from seesawfacenet_pytorch.src.seesaw_models import seesaw_shareFaceNet, seesaw_shuffleFaceNet, DW_seesawFaceNetv2
from PIL import Image
import cv2
import numpy as np
import torch.nn as nn
import onnx
from onnx_tf.backend import prepare
from torch.autograd import Variable
import tensorflow as tf


In [4]:
class CustomNet(nn.Module):
    def __init__(self, model):
        super(CustomNet, self).__init__()
        self.pre_process = T.Compose([T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
        self.backbone = model

    def forward(self,x):
        x = self.pre_process(x)
        return self.backbone(x)

def get_seesawnet_model(model, saved_dict):
    # saved_dict = torch.load("saved_models/Seesaw-shuffleFaceNet_160bs/save/model_2019-06-22-17-20_accuracy_0.9184285714285714_step_568064_final.pth", map_location="cuda")
    saved_dict = {v[7:]:k for v,k in saved_dict.items()}
    model.load_state_dict(saved_dict)
    model = model.cpu()
    model.eval()

    seesawnet_model = CustomNet(model)
    seesawnet_model.eval()

    img = Image.open("img.jpg")
    img = torch.FloatTensor(cv2.resize(np.asarray(img), (112, 112))).permute(2,0,1)
    img = img/255.
    img = img.unsqueeze(0)
    out = model(img)

    return seesawnet_model

reimplement_model_seesawnet = CustomNet(torch.jit.load("saved_models/Seesaw_shuffleFaceNet.pt", map_location="cpu")).cpu().eval()

models_configs = [
                 ("seesaw_shuffleFaceNet_160", seesaw_shuffleFaceNet.seesaw_shuffleFaceNet(512), "saved_models/Seesaw-shuffleFaceNet_160bs/save/model_2019-06-22-17-20_accuracy_0.9184285714285714_step_568064_final.pth"),
                 ("seesaw_shuffleFaceNet_192", seesaw_shuffleFaceNet.seesaw_shuffleFaceNet(512), "saved_models/Seesaw-shuffleFaceNet_192bs/save/model_2019-06-14-06-06_accuracy_0.9269999999999999_step_475328_final.pth"),
                 ("seesaw_shareFaceNet_160", seesaw_shareFaceNet.seesaw_shareFaceNet(512), "saved_models/Seesaw-shareFaceNet_160bs/save/model_2019-06-19-04-38_accuracy_0.9298571428571428_step_568064_final.pth"),
                 ("DW_seesawFaceNetv2", DW_seesawFaceNetv2.DW_seesawFaceNetv2(512), "saved_models/DW-SeesawFaceNetv2_128bs/save/model_2019-08-03-07-50_accuracy_0.9486000000000001_step_727840_final.pth")
]



for name,model,saved_dict_path in models_configs:
    print(name)
    if saved_dict_path == None:
        model = model

    else:
        model = get_seesawnet_model(model, torch.load(saved_dict_path))

    model = model.cpu()
    model.eval()


seesaw_shuffleFaceNet_160
seesaw_shuffleFaceNet_192
seesaw_shareFaceNet_160
DW_seesawFaceNetv2


In [9]:
for name,model,saved_dict_path in models_configs:
    print(name)
    if saved_dict_path == None:
        model = model

    else:
        model = get_seesawnet_model(model, torch.load(saved_dict_path))

    model = model.cpu()
    model.eval()

    dummy_input = Variable(torch.randn(1, 3, 112, 112))
    torch.onnx.export(model, dummy_input, f"saved_models/onnx/{name}.onnx")

    model = onnx.load(f"saved_models/onnx/{name}.onnx")
    tf_rep = prepare(model)
    tf_rep.export_graph(f"saved_models/tf/{name}.pb")

    # model = tf.saved_model.load(f"saved_models/tf/{name}.pb")

    converter = tf.lite.TFLiteConverter.from_saved_model(f"saved_models/tf/{name}.pb")
    tflite_model = converter.convert()

    # Save the model.
    with open(f'saved_models/lite/{name}.tflite', 'wb') as f:
      f.write(tflite_model)


seesaw_shuffleFaceNet_160


c:\Users\Gokul adethya\Desktop\SIH\SIH_dl_models\seesawfacenet_pytorch\src\seesaw_models\seesaw_shuffleFaceNet.py:230: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x1 = x[:, :(x.shape[1]//4), :, :]
c:\Users\Gokul adethya\Desktop\SIH\SIH_dl_models\seesawfacenet_pytorch\src\seesaw_models\seesaw_shuffleFaceNet.py:231: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b

INFO:tensorflow:Assets written to: saved_models/tf/seesaw_shuffleFaceNet_160.pb\assets


INFO:tensorflow:Assets written to: saved_models/tf/seesaw_shuffleFaceNet_160.pb\assets


seesaw_shuffleFaceNet_192


...

INFO:tensorflow:Assets written to: saved_models/tf/seesaw_shareFaceNet_160.pb\assets


DW_seesawFaceNetv2


c:\Users\Gokul adethya\Desktop\SIH\SIH_dl_models\seesawfacenet_pytorch\src\seesaw_models\DW_seesawFaceNetv2.py:233: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x1 = x[:, :(x.shape[1]//4), :, :]
c:\Users\Gokul adethya\Desktop\SIH\SIH_dl_models\seesawfacenet_pytorch\src\seesaw_models\DW_seesawFaceNetv2.py:234: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, roun

INFO:tensorflow:Assets written to: saved_models/tf/DW_seesawFaceNetv2.pb\assets


INFO:tensorflow:Assets written to: saved_models/tf/DW_seesawFaceNetv2.pb\assets


In [8]:
model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x2288a0f83a0>